In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from torch.nn.modules.conv import ConvTranspose2d
from torch.nn.modules import padding
class AutoEncoder(nn.Module):
    def __init__(self):

        super().__init__()

        self.encoder = nn.Sequential( # input : 3601*3601
            nn.Conv2d(1, 32, kernel_size=(15, 15), stride=4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2, 2), stride=2),
            nn.Conv2d(32, 64, kernel_size=(8, 8), stride=4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2, 2), stride=2),
            nn.Conv2d(64, 128, kernel_size=(4, 4), stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2, 2), stride=2),
        )

        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="bilinear"),
            nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=2, output_padding=1),
            nn.ReLU(),
            nn.Upsample(size=(111, 111), mode="bilinear"),
            nn.ConvTranspose2d(64, 32, kernel_size=(8, 8), stride=4, output_padding=0),
            nn.ReLU(),
            nn.Upsample(size=(897, 897), mode="bilinear"),
            nn.ConvTranspose2d(32, 1, kernel_size=(15, 15), stride=4, output_padding=2),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def dimensions(self, x):
        print(f"----------ENCODER----------")
        print(f"Input shape : {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[0](x)
        print(f"After 1st Conv2d: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[1](x)
        print(f"After 1st Maxpooling: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[2](x)
        print(f"After 1st ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[3](x)
        print(f"After 2nd Conv2d: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[4](x)
        print(f"After 2nd Maxpooling: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[5](x)
        print(f"After 2nd ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[6](x)
        print(f"After 3rd Conv2d: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[7](x)
        print(f"After 3rd Maxpooling: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[8](x)
        print(f"After 3rd ReLU: {x.shape[1], x.shape[2], x.shape[3]}")


        print("---------DECODER------------")
        x= self.decoder[0](x)
        print(f"After 1st Upsample: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[1](x)
        print(f"After 1st Transpose: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[2](x)
        print(f"After 1st ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[3](x)
        print(f"After 2nd Upsample: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[4](x)
        print(f"After 2nd Transpose: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[5](x)
        print(f"After 2nd ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[6](x)
        print(f"After 3rd Upsamle: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[7](x)
        print(f"After 3rd Transpose: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[8](x)
        print(f"Decoder Sigmoid : {x.shape[1], x.shape[2], x.shape[3]}")

In [4]:
load = torch.load(r'/content/drive/My Drive/Data/DEM_slope_tensorv2.pt')

In [5]:
class DEM_Dataset(Dataset):
    def __init__(self, data):
        self.filename = list(data.keys())
        self.data = data

    def __len__(self):
        return len(self.filename)

    def __getitem__(self, index):
        filename = self.filename[index]
        data = self.data[filename]
        return data

In [6]:
model = AutoEncoder()

In [7]:
dummy_input = torch.randn(5, 1, 3601, 3601)
model.dimensions(dummy_input)

----------ENCODER----------
Input shape : (1, 3601, 3601)
After 1st Conv2d: (32, 897, 897)
After 1st Maxpooling: (32, 897, 897)
After 1st ReLU: (32, 448, 448)
After 2nd Conv2d: (64, 111, 111)
After 2nd Maxpooling: (64, 111, 111)
After 2nd ReLU: (64, 55, 55)
After 3rd Conv2d: (128, 26, 26)
After 3rd Maxpooling: (128, 26, 26)
After 3rd ReLU: (128, 13, 13)
---------DECODER------------
After 1st Upsample: (128, 26, 26)
After 1st Transpose: (64, 55, 55)
After 1st ReLU: (64, 55, 55)
After 2nd Upsample: (64, 111, 111)
After 2nd Transpose: (32, 448, 448)
After 2nd ReLU: (32, 448, 448)
After 3rd Upsamle: (32, 897, 897)
After 3rd Transpose: (1, 3601, 3601)
Decoder Sigmoid : (1, 3601, 3601)


In [8]:
data = DEM_Dataset(load)
batch_size = 4
train_size = 35
test_size = 7
val_size = 8

train_dataset, test_dataset, val_dataset = random_split(data, [train_size, test_size, val_size])
#data_loader = DataLoader(data, batch_size=5, shuffle=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [21]:
data[0].shape

torch.Size([3601, 3601])

In [ ]:
model = AutoEncoder()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()  # Assuming you are using a linear activation for the final layer
optimizer = optim.Adam(model.parameters(), lr=1e-6)

num_epochs = 100
for epoch in tqdm(range(num_epochs), desc='Epochs'):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):

        batch_data = batch_data.float().to(device)
        optimizer.zero_grad()
        #print(batch_data.shape)
        outputs = model(batch_data.unsqueeze(1))
        loss = criterion(outputs, batch_data)
        loss.backward()
        optimizer.step()
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for val_batch_data in val_loader:
            val_batch_data = val_batch_data.float().to(device)
            val_outputs = model(val_batch_data.unsqueeze(1))
            val_loss += criterion(val_outputs, val_batch_data).item()

    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss}")

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4, 3601, 3601])) that is different to the input size (torch.Size([4, 1, 3601, 3601])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([3, 3601, 3601])) that is different to the input size (torch.Size([3, 1, 3601, 3601])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epochs:   1%|          | 1/100 [00:03<05:22,  3.26s/it]

Epoch 1/100, Loss: 0.15380047261714935, Val Loss: 0.15965285152196884


Epochs:   2%|▏         | 2/100 [00:06<04:51,  2.98s/it]

Epoch 2/100, Loss: 0.14710751175880432, Val Loss: 0.1596444770693779


Epochs:   3%|▎         | 3/100 [00:08<04:39,  2.88s/it]

...

Epoch 61/100, Loss: 0.20473001897335052, Val Loss: 0.15759281814098358


Epochs:  62%|██████▏   | 62/100 [02:56<01:46,  2.79s/it]

Epoch 62/100, Loss: 0.19150051474571228, Val Loss: 0.15743127465248108


Epochs:  63%|██████▎   | 63/100 [02:59<01:44,  2.84s/it]

Epoch 63/100, Loss: 0.19213879108428955, Val Loss: 0.1572612300515175


In [25]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
with torch.no_grad():
    with tqdm(test_loader, desc='Testing', leave=False) as t:
        for test_batch_data in t:
            test_batch_data = test_batch_data.float().to(device)
            test_outputs = model(test_batch_data.unsqueeze(1))
            test_loss += criterion(test_outputs, test_batch_data).item()

test_loss /= len(test_loader)  # Average the test loss
print(f"Test Loss: {test_loss}")

Testing:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4, 3601, 3601])) that is different to the input size (torch.Size([4, 1, 3601, 3601])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Testing:  50%|█████     | 1/2 [00:00<00:00,  3.23it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([3, 3601, 3601])) that is different to the input size (torch.Size([3, 1, 3601, 3601])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
                                                      

Test Loss: 0.1148299127817154


In [ ]:
#torch.save(embedded_dict, "/home/smitesh22/Unsupervised-Machine-Learning-for-Solar-Site-Selection/Code/embedded_dict.pt")

torch.save(model.state_dict(), "DEM_model_weights.pt")